# Sentiment Analysis

Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import spacy
import collections
import operator
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Dataset

Loading the [Sentiment140](https://www.kaggle.com/kazanova/sentiment140) Dataset that includes 1.6 milion tweets

In [2]:
%%time
columns = ["target", "ids", "date", "flag", "user", "text"]
encoding = "ISO-8859-1"
dataset = pd.read_csv('dataset/tweets.csv', encoding=encoding, names=columns)

CPU times: user 3.16 s, sys: 339 ms, total: 3.5 s
Wall time: 4.18 s


Dropping columns with not useful data

In [3]:
dataset.drop(columns=["ids", "date", "flag", "user"], inplace=True)
dataset.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


Since the original dataset is using classes `0-Negative` and `4-Positive`, I am mapping the `4` to `1` to fix the confusing naming

In [4]:
dataset.target.replace({4: 1}, inplace=True)

Now lets shuffle the rows in dataset, since originally they are sorted by the class

In [5]:
dataset = dataset.sample(frac=1).reset_index(drop=True)
# Remove for final training
dataset = dataset.iloc[:120_000]

In [6]:
dataset.head()

,target,text
0,1,@darkdaisy Yah. I actually am going. I booked ...
1,1,School's out! Let the mayhem commence! Ideally...
2,0,Now Jump Up Let's Get Krazyyy! In such a part...
3,1,@LMsouthgate laaura do u know? moises arias ta...
4,0,I wish I was still sleeping


### Preprocessing

Dataset is now ready for preprocessing. I will use [spacy](https://spacy.io) which is a very popular library for Natural Language Processing.

Firstly, lets load a model that will help us with tokenisation and lemmatisation of tweets included in the dataset

In [7]:
%%time
nlp = spacy.load("en_core_web_lg")

CPU times: user 6.87 s, sys: 1.44 s, total: 8.3 s
Wall time: 19.7 s


Piping through the tweets from dataset, will result in a iterator object that we will need to use to iterate over preprocessed tweets - and tokenized words in each tweet.

In [8]:
tweets_iterator = nlp.pipe(dataset.text, n_threads=-1, batch_size=32)

Lets find out which are the most popular words in the dataset by mapping them to a dictionary. This can be useful during the optimalisation process, if we would want to drop some rare words to not include them in the final model to reduce its size. This process can take quite a while, since we are operating on a very large dataset.

In [9]:
words_to_lexeme = {}
words = collections.defaultdict(int)
preprocessed_text = []
for tweet in tqdm(tweets_iterator, total=dataset.shape[0]):
    preprocessed_tweet = []
    for token in tweet:
        if token.is_stop:
            continue
        lexeme = nlp.vocab[token.lemma]
        if lexeme.has_vector:
            words_to_lexeme[lexeme.text.lower()] = lexeme
            words[lexeme.text.lower()] += 1
            preprocessed_tweet.append(lexeme.text)
    preprocessed_text.append(preprocessed_tweet)

Now lets update our dataset with new column that will contain preprocessed tweet contents

In [10]:
dataset['preprocessed'] = preprocessed_text
dataset.head()

,target,text,preprocessed
0,1,@darkdaisy Yah. I actually am going. I booked ...,"[yah, ., actually, go, ., book, ., Alaska, ., ..."
1,1,School's out! Let the mayhem commence! Ideally...,"[school, !, let, mayhem, commence, !, ideally,..."
2,0,Now Jump Up Let's Get Krazyyy! In such a part...,"[jump, let, !, party, mood, ., bad, ground, .]"
3,1,@LMsouthgate laaura do u know? moises arias ta...,"[u, know, ?, moise, aria, talk, mee, !, !]"
4,0,I wish I was still sleeping,"[wish, sleep]"


Sorting the counts, with its corresponding tokens

In [12]:
sorted_words = sorted(words.items(), key=operator.itemgetter(1), reverse=True)
print('Top 10 words are:')
_ = [print(word, count) for word, count in sorted_words[:10]]

Top 10 words are:
! 68007
. 60375
, 36121
? 17951
... 16535
- 8877
go 8753
good 8160
day 7935
get 7315


Now, once we have the tokens, sorted by their popularity, we will need to create the `embedding_matrix`. The matrix consists of vectors for each token/word that we get from spacy. One important thing to notice is that we will make an extra entry in the matrix at index 0, that will represent the placeholder for empty word. To train the model (and to recieve the prediction) we will feed it with array of ids of words from the tweet. These array will need to be equal lenght, because neural network will always expect the input to be equal size. We will use this row `0` to refer to a empty token, so we can pad the input array to match the expected by neural network shape of the input. 

Also I will create a `word_index` array to keep the reference of word to index so we can convert that later for proper neural network input.

In [20]:
word_index = {'': 0}
embedding_matrix = np.zeros((len(words) + 1, 300))

for i, word in enumerate(words, start=1):
    word_index[word] = i
    embedding_matrix[i] = words_to_lexeme[word].vector

The resulting matrix is a very important part of the model, since it will serve as a weights matrix in the first layer of the model.

In [21]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.32356   , -0.097893  , -0.33511999, ...,  0.2814    ,
        -0.30553001, -0.089444  ],
       [ 0.012001  ,  0.20750999, -0.12578   , ...,  0.13871001,
        -0.36048999, -0.035     ],
       ...,
       [ 0.053052  ,  0.13817   , -0.51157999, ...,  0.39816001,
         0.21936999, -0.094792  ],
       [ 0.34850001, -0.56590998, -0.1804    , ...,  0.79347003,
         0.33541   , -0.096093  ],
       [ 0.0069292 , -0.64995998, -0.43669   , ..., -0.22797   ,
         0.30250001, -0.23746   ]])

### Model

At this point, once we are ready with the preprocessing data, lets design the architecture for our model. I will use [Keras](https://keras.io), which is a very popular Deep Learning library.

The model is a very simple Convolutional Neural Network consisting of 9 layers.

In [22]:
model = keras.Sequential([
    keras.layers.Embedding(len(top_embedding_matrix), 300, weights=[top_embedding_matrix], input_length=100),
    keras.layers.Dropout(0.5),
    keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          10971300  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 11,131,801
Trainable params: 11,131,801
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(
    loss='binary_crossentropy', optimizer="adam", metrics=['accuracy']
)

### Training

This is the part where we jump to the training our model. Lets split out data into training and testing subsets of the original dataset.

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    dataset.preprocessed, dataset.target, test_size=0.2, random_state=42
)

Before we will pipe our data into the model, there is one last thing to do. We need to map the words form tweet to ids, and to fill the array with zeros so the model will accept it as the input

In [25]:
def map_words(row):
    return np.array([word_index[word.lower()] if word.lower() in word_index else 0 for word in row])

In [26]:
X_train = X_train.apply(map_words)
X_test = X_test.apply(map_words)
X_train = pad_sequences(X_train, maxlen=100)
X_test = pad_sequences(X_test, maxlen=100)

In [27]:
print('Train data shapes:', 'X', X_train.shape, 'y', y_train.shape)
print('Test data shapes:', 'X', X_test.shape, 'y', y_test.shape)

Train data shapes: X (96000, 100) y (96000,)
Test data shapes: X (24000, 100) y (24000,)


Ok, finally! Let's go for it!

In [28]:
epochs = 5
batch_size = 64

In [29]:
model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    verbose=1
)

Instructions for updating:
Use tf.cast instead.
Train on 86400 samples, validate on 9600 samples
Epoch 1/5
86400/86400 [==============================] - 423s 5ms/step - loss: 0.5250 - acc: 0.7343 - val_loss: 0.5005 - val_acc: 0.7551
Epoch 2/5
86400/86400 [==============================] - 408s 5ms/step - loss: 0.4496 - acc: 0.7895 - val_loss: 0.5046 - val_acc: 0.7511
Epoch 3/5
86400/86400 [==============================] - 401s 5ms/step - loss: 0.4016 - acc: 0.8160 - val_loss: 0.5200 - val_acc: 0.7479
Epoch 4/5
86400/86400 [==============================] - 448s 5ms/step - loss: 0.3567 - acc: 0.8393 - val_loss: 0.5593 - val_acc: 0.7427
Epoch 5/5
86400/86400 [==============================] - 481s 6ms/step - loss: 0.3153 - acc: 0.8569 - val_loss: 0.6116 - val_acc: 0.7332


### Testing

In [30]:
model.evaluate(X_test, y_test, batch_size=batch_size)

24000/24000 [==============================] - 16s 650us/step


[0.6055881338914235, 0.73875]

Lets run the model on some custom tweets

In [34]:
def preprocessed(tweets):
    preprocessed = []
    for tweet in nlp.pipe(tweets, n_threads=-1, batch_size=32):
        preprocessed_tweet = []
        for token in tweet:
            if token.is_stop:
                continue
            lexeme = nlp.vocab[token.lemma]
            if lexeme.has_vector:
                preprocessed_tweet.append(lexeme.text)
        preprocessed.append(preprocessed_tweet)
    preprocessed = [map_words(tweet) for tweet in preprocessed]
    return pad_sequences(preprocessed, maxlen=100)[0].reshape(1, 100)

In [36]:
tweets = [
    'Devastated to hear about the lives lost in Lee County, a place close to my heart. Wishing strength and healing for all those affected by today’s tornadoes.',
    'Impressive work by these young photographers! Using iPhone XR, more than 260 schoolchildren from around Chennai, India captured images of their community for the annual #ChennaiPhotoBiennale. #ShotoniPhone',
    'Saddened to hear of Pierre Nanterme’s passing. He will be remembered for his contributions to business as well as his work toward equality in the workplace. Our deepest condolences go out to his friends, family and everyone at Accenture.',
    'Steve’s vision is reflected all around us at Apple Park. He would have loved it here, in this place he dreamed up — the home and inspiration for Apple’s future innovations. We miss him today on his 64th birthday, and every day.'
]

for tweet in tweets:
    preped = preprocessed([tweet])
    print('Tweet :', tweet)
    print('Sentiment :', model.predict(preped)[0][0], '\n')

Tweet : Devastated to hear about the lives lost in Lee County, a place close to my heart. Wishing strength and healing for all those affected by today’s tornadoes.
Sentiment : 0.0011292003 

Tweet : Impressive work by these young photographers! Using iPhone XR, more than 260 schoolchildren from around Chennai, India captured images of their community for the annual #ChennaiPhotoBiennale. #ShotoniPhone
Sentiment : 0.045893963 

Tweet : Saddened to hear of Pierre Nanterme’s passing. He will be remembered for his contributions to business as well as his work toward equality in the workplace. Our deepest condolences go out to his friends, family and everyone at Accenture.
Sentiment : 0.04379572 

Tweet : Steve’s vision is reflected all around us at Apple Park. He would have loved it here, in this place he dreamed up — the home and inspiration for Apple’s future innovations. We miss him today on his 64th birthday, and every day.
Sentiment : 0.2523569 



### Export

After we are done with training and our model is ready, we have to export the model to the file so I can convert it to `.mlmodel` with `coremltools`.

In [ ]:
model.save('SentimentalAnalysisModel.h5')

Also the word index needs to be exported and a simple txt file will be just enough. The line number will serve as id

In [ ]:
with open("words.txt", "w") as text_file:
    for key in word_index.keys():
        text_file.write(key + '\n')